In [68]:
# Caregar llibreries
import pandas as pd
import requests
from datetime import datetime, timedelta
import numpy as np

Descargar datos de temperatura y humedad de los últimos 7 días

In [ ]:
# URL del recurso
url1 = "https://analisi.transparenciacatalunya.cat/resource/nzvn-apee.json"

# Fecha hace 7 días
hace_21_dias = (datetime.today() - timedelta(days=21)).strftime('%Y-%m-%dT00:00:00')

# Parámetros: columnas válidas + fecha de filtro
params1 = {
    "$where": f"data_lectura >= '{hace_21_dias}' AND (codi_variable = '32' OR codi_variable = '33')",
    "$select": "id,codi_variable,data_lectura,valor_lectura",
    "$order": "data_lectura DESC",
    "$limit": 500000
}

# Descargar y procesar
response1 = requests.get(url1, params=params1)
response1.raise_for_status()

               id codi_variable data_lectura  valor_lectura
0  Z7330405250000            33   2025-05-04           86.0
1  Z9320405250000            32   2025-05-04            6.6
2  Z9330405250000            33   2025-05-04           84.0
3  ZB320405250000            32   2025-05-04            4.2
4  ZB330405250000            33   2025-05-04           98.0


TypeError: 'Series' object is not callable

In [ ]:
# Convertir variables
df1 = pd.DataFrame(response1.json())
df1['data_lectura'] = pd.to_datetime(df1['data_lectura'])
df1['valor_lectura'] = pd.to_numeric(df1['valor_lectura'])

print(df1.head())
print(df1.dtypes)

               id codi_variable data_lectura  valor_lectura
0  Z7330405250000            33   2025-05-04           86.0
1  Z9320405250000            32   2025-05-04            6.6
2  Z9330405250000            33   2025-05-04           84.0
3  ZB320405250000            32   2025-05-04            4.2
4  ZB330405250000            33   2025-05-04           98.0
id                       object
codi_variable            object
data_lectura     datetime64[ns]
valor_lectura           float64
dtype: object


In [44]:
# Separate the df1 by temperature and humidity
temperature = df1[df1['codi_variable'] == '32']
temperature = temperature.drop(columns=['codi_variable', 'id'])
temperature

humidity = df1[df1['codi_variable'] == '33']
humidity = humidity.drop(columns=['codi_variable', 'id'])
humidity

,data_lectura,valor_lectura
0,2025-05-04,86.0
2,2025-05-04,84.0
4,2025-05-04,98.0
6,2025-05-04,88.0
8,2025-05-04,100.0
...,...,...
356664,2025-04-13,100.0
356666,2025-04-13,99.0
356668,2025-04-13,100.0
356671,2025-04-13,100.0


In [ ]:
# Mean the temperature daily
temperature_daily = temperature.groupby('data_lectura').mean().reset_index()
humidity_daily = humidity.groupby('data_lectura').mean().reset_index()
humidity_daily 

,data_lectura,valor_lectura
0,2025-04-13 00:00:00,96.360656
1,2025-04-13 00:30:00,96.622951
2,2025-04-13 01:00:00,96.748634
3,2025-04-13 01:30:00,97.081967
4,2025-04-13 02:00:00,97.382514
...,...,...
999,2025-05-03 22:00:00,91.033149
1000,2025-05-03 22:30:00,91.453039
1001,2025-05-03 23:00:00,92.497238
1002,2025-05-03 23:30:00,93.251397


Descargar datos de infecciones respiratorias de los últimos 7 días

In [47]:
# URL del recurso
url2 = 'https://analisi.transparenciacatalunya.cat/resource/fa7i-d8gc.json'

# Parámetros: columnas válidas + fecha de filtro
params2 = {
    "$where": f"data >= '{hace_21_dias}'",
    "$select": "data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos",
    "$order": "data DESC",
    "$limit": 500000
}

# Descargar y procesar
response2 = requests.get(url2, params=params2)
response2.raise_for_status()


In [63]:
# Convertir variables
df2 = pd.DataFrame(response2.json())
df2['data'] = pd.to_datetime(df2['data'])
df2['casos'] = pd.to_numeric(df2['casos'])
df2['setmana_epidemiologica'] = pd.to_numeric(df2['setmana_epidemiologica'])


print(df2.dtypes)
df2

data                      datetime64[ns]
setmana_epidemiologica             int64
any                               object
nom_regio                         object
diagnostic                        object
grup_edat                         object
casos                              int64
dtype: object


,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos
0,2025-04-27,17,2025,Barcelona Metropolitana Sud,Faringoamigdalitis,5 a 9,1
1,2025-04-27,17,2025,Camp de Tarragona,Grip,50 a 54,1
2,2025-04-27,17,2025,Barcelona Ciutat,Faringoamigdalitis,25 a 29,1
3,2025-04-27,17,2025,Barcelona Ciutat,Altres IRA,80 o més,1
4,2025-04-27,17,2025,Barcelona Metropolitana Nord,Altres IRA,55 a 59,1
...,...,...,...,...,...,...,...
76265,2025-04-13,15,2025,Lleida,Faringoamigdalitis,1 i 2,1
76266,2025-04-13,15,2025,Girona,Altres IRA,35 a 39,1
76267,2025-04-13,15,2025,Barcelona Metropolitana Sud,Altres IRA,45 a 49,1
76268,2025-04-13,15,2025,Barcelona Metropolitana Sud,Altres IRA,15 a 19,1


In [64]:
# group by 
df2 = df2.groupby(['data', 'setmana_epidemiologica', 'any', 'nom_regio', 'diagnostic', 'grup_edat'], as_index=False).agg({'casos': 'sum'})
df2

,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos
0,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,1 i 2,3
1,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,15 a 19,3
2,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,20 a 24,1
3,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,3 i 4,2
4,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,30 a 34,1
...,...,...,...,...,...,...,...
11484,2025-04-27,17,2025,Terres de l'Ebre,Grip,35 a 39,1
11485,2025-04-27,17,2025,Terres de l'Ebre,Grip,5 a 9,1
11486,2025-04-27,17,2025,Terres de l'Ebre,Grip,55 a 59,1
11487,2025-04-27,17,2025,Terres de l'Ebre,Grip,65 a 69,1


In [65]:
# merge temperature, humidity and df using 'data' and 'DATA_LECTURA' as keys
df2 = df2.merge(temperature_daily, left_on='data', right_on='data_lectura', how='left')
df2 = df2.merge(humidity_daily, left_on='data', right_on='data_lectura', how='left')
df2 = df2.drop(columns=['data_lectura_x', 'data_lectura_y'])
df2 = df2.rename(columns={'valor_lectura_x': 'temperature', 'valor_lectura_y': 'humidity'})
df2


,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos,temperature,humidity
0,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,1 i 2,3,11.363934,96.360656
1,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,15 a 19,3,11.363934,96.360656
2,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,20 a 24,1,11.363934,96.360656
3,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,3 i 4,2,11.363934,96.360656
4,2025-04-13,15,2025,Alt Pirineu i Aran,Altres IRA,30 a 34,1,11.363934,96.360656
...,...,...,...,...,...,...,...,...,...
11484,2025-04-27,17,2025,Terres de l'Ebre,Grip,35 a 39,1,9.449162,87.916201
11485,2025-04-27,17,2025,Terres de l'Ebre,Grip,5 a 9,1,9.449162,87.916201
11486,2025-04-27,17,2025,Terres de l'Ebre,Grip,55 a 59,1,9.449162,87.916201
11487,2025-04-27,17,2025,Terres de l'Ebre,Grip,65 a 69,1,9.449162,87.916201


In [71]:
# Converit semana
df2['semana_sin'] = np.sin(2 * np.pi * df2['setmana_epidemiologica'] / 53)
df2['semana_cos'] = np.cos(2 * np.pi * df2['setmana_epidemiologica'] / 53)